In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Đọc dữ liệu
data = pd.read_csv('C:\ML_Data\Dataset_Vietnam\Yen Bai.csv', parse_dates=['date'])
data.set_index('date', inplace=True)
data = data.sort_index()

# Tạo biến đầu vào (X) và biến mục tiêu (y)
X = data[['cloud', 'max', 'rain']]
y = data['humidi']

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Gán lại index cho y_test để đảm bảo chỉ số đúng
y_test = pd.Series(y_test, index=X_test.index)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Tạo pipeline với Polynomial Features và Linear Regression
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('lr', LinearRegression())
])

# Định nghĩa tham số cần tìm kiếm
param_grid = {
    'poly__degree': [1, 2, 3, 4],  # Độ bậc đa thức
    'lr__fit_intercept': [True, False]
}

# Grid Search
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
best_params = grid_search.best_params_
print("Best Parameters for Linear Regression:", best_params)

# Dự đoán với mô hình tốt nhất
best_lr = grid_search.best_estimator_
y_pred = best_lr.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
# Chọn khoảng thời gian hiển thị
start_date = y_test.index[-30]  # 30 ngày cuối trong tập kiểm tra
end_date = y_test.index[-1]

# Lọc dữ liệu cho khoảng thời gian này
y_test_filtered = y_test.loc[start_date:end_date]
y_pred_filtered = pd.Series(y_pred, index=y_test.index).loc[start_date:end_date]

# Vẽ biểu đồ
plt.figure(figsize=(12, 6))
plt.plot(y_test_filtered.index, y_test_filtered, label='Giá trị thực tế', color='blue', linewidth=2)
plt.plot(y_pred_filtered.index, y_pred_filtered, label='Giá trị dự đoán', color='red', linestyle='--', linewidth=2)
plt.title(f'So sánh giá trị thực tế và dự đoán ({start_date.date()} đến {end_date.date()})', fontsize=16)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Độ ẩm (Humidity)', fontsize=12)
plt.legend(fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()